In [1]:
import time
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
import random
import warnings

In [2]:
sales = pd.read_csv('https://raw.githubusercontent.com/hoganj15/MMA_Assignment_Data/main/Service/data_processed.csv') 
sales.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_2,functionality_3,functionality_4,functionality_5,functionality_6,functionality_7,functionality_8,functionality_9,functionality_10,functionality_11,functionality_12,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


Optimal decentralized model

Train-test split function for time series (splices input arrays based on user-defined test set size)

In [3]:
def train_test_split(X, y, test_size): #custom train_test_split function
  X_train, X_test = np.split(X, [int(len(X)*(1-test_size))])
  y_train, y_test = np.split(y, [int(len(y)*(1-test_size))])
  return X_train, X_test, y_train, y_test

Defining the hyperparameter search spaces for KNN regressor and XGBoost regressor, and setting up the tuning model for Bayesian hyperparameter optimization. We keep the search spaces pretty broad because Bayesian optimization will hone in on regions of high performance and optimize around there. We want to provide an ample number of such spaces.

In [4]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 0,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

def tune_model(space): 
    xgb_model = xgb.XGBRegressor(n_estimators =space['n_estimators'], 
                                 max_depth = int(space['max_depth']), 
                                 gamma = space['gamma'],
                                 reg_alpha = int(space['reg_alpha']),
                                 min_child_weight=int(space['min_child_weight']),
                                colsample_bytree=int(space['colsample_bytree']),
                                 verbosity=0)
    
    xgb_model.fit(X_train, y_train,
            eval_set=[(X_train, y_train), (X_test, y_test)],
            early_stopping_rounds=50,
            verbose=False) 

    pred = xgb_model.predict(X_test)
    #pred = pred.reshape(len(pred),1)
    R2 = r2_score(y_test, pred)
    return {'loss': -1*R2, 'status': STATUS_OK} #trying to minimize -R2 is the same as maximizing R2

In [5]:
knn_space = {
    'n_neighbors': hp.uniform('n_neighbors', 2, 25),
    'weights': hp.choice('weights', ['uniform', 'distance']),
    'algorithm': hp.choice('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
    'leaf_size': hp.uniform('leaf_size', 5, 90)
}

weights_values = {0: 'uniform', 1: 'distance'}
algorithm_values = {0: 'auto', 1: 'ball_tree', 2: 'kd_tree', 3: 'brute'}

def tune_knn_model(space): 
    knn_model = KNeighborsRegressor(n_neighbors=int(space['n_neighbors']), weights=space['weights'], algorithm=space['algorithm'], leaf_size=int(space['leaf_size']))
    knn_model.fit(X_train, y_train)

    pred = knn_model.predict(X_test)
    #pred = pred.reshape(len(pred),1)
    R2 = r2_score(y_test, pred)
    return {'loss': -1*R2, 'status': STATUS_OK} #trying to minimize -R2 is the same as maximizing R2

Loops through each SKU and optimizes an XGBoost regressor, KNN regressor, and ridge regression model, then returns the one with the highest R2 value for that SKU.

In [6]:
np.random.seed(6)
best_models = dict()
y_test_total = []
y_test_pred = []

start = time.time()
skus = set(sales['sku'])

for sku in skus:
  max_r2 = -100 #initializing best R2 for that sku
  df_sku = sales[sales['sku'] == sku]
  X = df_sku.drop(['week', 'sku', 'weekly_sales'], axis = 1)
  y = df_sku[['weekly_sales']]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  
  y_test_total += list(y_test.values)

  #XGBoostRegressor
  trials = Trials()

  best_hyperparams = fmin(fn = tune_model,
                          space = space,
                          algo = tpe.suggest,
                          max_evals = 75,
                          trials = trials) #Bayesian optimization for every SKU
  best_hyperparams['max_depth'] = int(best_hyperparams['max_depth'])
  best_hyperparams['verbosity'] = 0
  xgb_model = xgb.XGBRegressor(**best_hyperparams)
  xgb_model.fit(X_train, y_train, eval_set = [(X_train, y_train), (X_test, y_test)], early_stopping_rounds=50, verbose=False)
  pred = xgb_model.predict(X_test)
  if r2_score(y_test, pred) > max_r2:
    max_r2 = r2_score(y_test, pred)
    y_pred_best = pred.reshape(-1, 1)
    best_models[sku] = xgb_model

  #KNNRegressor
  trials = Trials()

  best_hyperparams = fmin(fn = tune_knn_model,
                          space = knn_space,
                          algo = tpe.suggest,
                          max_evals = 75,
                          trials = trials) #Bayesian optimization for every SKU
  best_hyperparams['n_neighbors'] = int(best_hyperparams['n_neighbors'])
  best_hyperparams['leaf_size'] = int(best_hyperparams['leaf_size'])
  best_hyperparams['weights'] = weights_values[best_hyperparams['weights']]
  best_hyperparams['algorithm'] = algorithm_values[best_hyperparams['algorithm']]

  knn_model = KNeighborsRegressor(**best_hyperparams)
  knn_model.fit(X_train, y_train)
  pred = knn_model.predict(X_test)
  if r2_score(y_test, pred) > max_r2:
    max_r2 = r2_score(y_test, pred)
    y_pred_best = pred
    best_models[sku] = knn_model

  #RidgeCV
  ridge_model = linear_model.RidgeCV(alphas = [random.uniform(0, 2.5) for i in range(100)], scoring = 'r2') #RidgeCV takes a set of alphas, tries them all, and returns the best one using R2 as the metric
  ridge_model.fit(X_train, y_train)

  pred = ridge_model.predict(X_test)
  if r2_score(y_test, pred) > max_r2:
    max_r2 = r2_score(y_test, pred)
    y_pred_best = pred
    best_models[sku] = ridge_model

  y_test_pred += list(y_pred_best)
  print(f'SKU {sku} done')
print(f'Time to train: {time.time() - start} seconds')

100%|██████████| 75/75 [00:01<00:00, 74.26it/s, best loss: 4.64857256723581]
SKU 1 done
100%|██████████| 75/75 [00:00<00:00, 80.00it/s, best loss: 0.6072524561891064]
SKU 2 done
100%|██████████| 75/75 [00:00<00:00, 75.52it/s, best loss: 2.607695061031687]
SKU 3 done
100%|██████████| 75/75 [00:00<00:00, 87.67it/s, best loss: 2.1048644047184726]
SKU 4 done
100%|██████████| 75/75 [00:00<00:00, 80.89it/s, best loss: 1.4203182354483483]
SKU 5 done
100%|██████████| 75/75 [00:00<00:00, 81.92it/s, best loss: 0.42893400227257183]
SKU 6 done
100%|██████████| 75/75 [00:00<00:00, 82.50it/s, best loss: 0.04413529784789749]
SKU 7 done
100%|██████████| 75/75 [00:00<00:00, 82.24it/s, best loss: -0.3113105220942102]
SKU 8 done
100%|██████████| 75/75 [00:00<00:00, 78.40it/s, best loss: -0.13507130588827998]
SKU 9 done
100%|██████████| 75/75 [00:01<00:00, 73.69it/s, best loss: 0.678272785946487]
SKU 10 done
100%|██████████| 75/75 [00:01<00:00, 73.82it/s, best loss: -0.1594659418417954]
SKU 11 done
100%|█

In [7]:
print('Out of sample R2:',round(r2_score(np.array(y_test_total), np.array(y_test_pred)),3))
print('MSE:', round(mean_squared_error(np.array(y_test_total), np.array(y_test_pred)),3))

Out of sample R2: 0.667
MSE: 36915.227
